In [1]:
working_directory = '/mnt/DataRAID/melismail/PDAC'
import os
os.chdir(working_directory)
from pickle_utils import write_pickle, read_pickle

import pandas as pd
import numpy as np
import sys, cv2

from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import img_to_array

In [2]:
base_path = '/mnt/DataRAID/melismail/PDAC/data'
preprocessing_path ='Preprocessing_images'
filtered_path = 'filtered'
dim = 512
dim_path = f"{dim}x{dim}"
features_path = 'Features_extraction'
model_path = "InceptionV3"

In [3]:
filtered_img_dict = read_pickle(os.path.join(base_path, preprocessing_path, filtered_path, dim_path, f"{filtered_path}_{dim_path}_img_dict.pkl"))

In [4]:
# Function to Extract features from the images
def image_feature(img, model):
    x = np.expand_dims(img,axis=0)
    x = preprocess_input(x)
    feat=model.predict(x)
    feat=feat.flatten()
    return feat

In [5]:
features_img_dict = read_pickle(path=os.path.join(base_path, features_path, model_path, dim_path, f"{model_path}_{dim_path}_img_dict.pkl"))
if features_img_dict is None:    
    model = InceptionV3(weights='imagenet', include_top=False, input_shape=(dim,dim,3), pooling='avg')

    features_img_dict = {k: image_feature(img=v, model=model)
                        for k, v in filtered_img_dict.items()}

    write_pickle(path=os.path.join(base_path, features_path, model_path, dim_path, f"{model_path}_{dim_path}_img_dict.pkl"), obj=features_img_dict)
    print(f"Count of TILE features:  {len(features_img_dict.items())}")
else:
    print("Read from Disk")

path did not exist: /mnt/DataRAID/melismail/PDAC/data/Features_extraction/InceptionV3/512x512/InceptionV3_512x512_img_dict.pkl


2022-09-17 22:42:33.350513: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-17 22:42:33.858768: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 30988 MB memory:  -> device: 0, name: Quadro GV100, pci bus id: 0000:65:00.0, compute capability: 7.0
2022-09-17 22:42:37.208860: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8100
2022-09-17 22:42:37.609068: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-09-17 22:42:37.609908: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-09-1

Count of TILE features:  6349


In [6]:
model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 512, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 255, 255, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 255, 255, 32  96         ['conv2d[0][0]']                 
 alization)                     )                                                      

In [7]:
feature_df = pd.DataFrame(data=[{"Pseudonym": k[0], "tile_id": k[1], "Features": v} for k, v in features_img_dict.items()])
write_pickle(path=os.path.join(base_path, features_path, model_path, dim_path, f"{model_path}_{dim_path}_img_df.pkl"), obj=feature_df)